In [403]:
import pandas as pd
import numpy as np
import re

data = pd.read_json("AllCards.json")
data = data.T
data.index = range(len(data))
data.head()

,cmc,colorIdentity,colors,hand,imageName,layout,life,loyalty,manaCost,mciNumber,name,names,power,starter,subtypes,supertypes,text,toughness,type,types
0,6,"[R, G]","[Red, Green]",NaN,"ach! hans, run!",normal,NaN,NaN,{2}{R}{R}{G}{G},NaN,"""Ach! Hans, Run!""",NaN,NaN,NaN,NaN,NaN,"At the beginning of your upkeep, you may say ""...",NaN,Enchantment,[Enchantment]
1,5,"[W, U, B, R, G]","[White, Blue, Black, Red, Green]",NaN,world champion,normal,NaN,NaN,{W}{U}{B}{R}{G},NaN,1996 World Champion,NaN,*,NaN,[Legend],NaN,Cannot be the targets of spells or effects. Wo...,*,Summon — Legend,[Creature]
2,NaN,NaN,NaN,NaN,a display of my dark power,scheme,NaN,NaN,NaN,NaN,A Display of My Dark Power,NaN,NaN,NaN,NaN,NaN,"When you set this scheme in motion, until your...",NaN,Scheme,[Scheme]
3,3,[W],[White],NaN,awol,normal,NaN,NaN,{2}{W},NaN,AWOL,NaN,NaN,NaN,NaN,NaN,Remove target attacking creature from the game...,NaN,Instant,[Instant]
4,2,[B],[Black],NaN,abandon hope,normal,NaN,NaN,{X}{1}{B},1,Abandon Hope,NaN,NaN,NaN,NaN,NaN,"As an additional cost to cast Abandon Hope, di...",NaN,Sorcery,[Sorcery]


In [404]:
data.shape

(15884, 20)

In [405]:
data = data[data['cmc']>=0]
data = data[data['cmc']<100]
len(data)

15018

In [406]:
def hasItem(c,x):
    if type(x) is not list and np.isnan(x):
        return False
    if (c in x):
        return True
    else:
        return False

for col in ['White','Black','Red','Green','Blue']:
    data[col.lower()] = [1 if hasItem(col,x) else 0 for x in data['colors']]

In [407]:
del data['colorIdentity']
del data['colors']
del data['hand']
del data['imageName']
del data['names']
del data['mciNumber'] #apparently this is a phone card number... 
#when wizards of the coast worked with a company called mci prepaid.
del data['supertypes']
del data['life']
data.head()

,cmc,layout,loyalty,manaCost,name,power,starter,subtypes,text,toughness,type,types,white,black,red,green,blue
0,6,normal,NaN,{2}{R}{R}{G}{G},"""Ach! Hans, Run!""",NaN,NaN,NaN,"At the beginning of your upkeep, you may say ""...",NaN,Enchantment,[Enchantment],0,0,1,1,0
1,5,normal,NaN,{W}{U}{B}{R}{G},1996 World Champion,*,NaN,[Legend],Cannot be the targets of spells or effects. Wo...,*,Summon — Legend,[Creature],1,1,1,1,1
3,3,normal,NaN,{2}{W},AWOL,NaN,NaN,NaN,Remove target attacking creature from the game...,NaN,Instant,[Instant],1,0,0,0,0
4,2,normal,NaN,{X}{1}{B},Abandon Hope,NaN,NaN,NaN,"As an additional cost to cast Abandon Hope, di...",NaN,Sorcery,[Sorcery],0,1,0,0,0
6,4,normal,NaN,{3}{B},Abattoir Ghoul,3,NaN,[Zombie],First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],0,1,0,0,0


In [408]:
np.unique(data['power'])

array([nan, u'*', u'*\xb2', u'-1', u'.5', u'0', u'1', u'1+*', u'1.5',
       u'10', u'11', u'12', u'13', u'15', u'2', u'2+*', u'2.5', u'3',
       u'3.5', u'4', u'5', u'6', u'7', u'8', u'9', u'99'], dtype=object)

In [409]:
data['manaCost'].head()

0    {2}{R}{R}{G}{G}
1    {W}{U}{B}{R}{G}
3             {2}{W}
4          {X}{1}{B}
6             {3}{B}
Name: manaCost, dtype: object

In [410]:
sum([int(x) if x.isnumeric() else 0 for x in costList])

3

In [411]:
data['redCost'] = [x.count('R') for x in data['manaCost']]
data['blueCost'] = [x.count('U') for x in data['manaCost']]
data['greenCost'] = [x.count('G') for x in data['manaCost']]
data['blackCost'] = [x.count('B') for x in data['manaCost']]
data['whiteCost'] = [x.count('W') for x in data['manaCost']]
data['xCost'] = [x.count('X') for x in data['manaCost']]
data['colorlessCost'] = [sum([int(x) if x.isnumeric() else 0 for x in costList]) for costList in data['manaCost']]
data.head()

,cmc,layout,loyalty,manaCost,name,power,starter,subtypes,text,toughness,...,red,green,blue,redCost,blueCost,greenCost,blackCost,whiteCost,xCost,colorlessCost
0,6,normal,NaN,{2}{R}{R}{G}{G},"""Ach! Hans, Run!""",NaN,NaN,NaN,"At the beginning of your upkeep, you may say ""...",NaN,...,1,1,0,2,0,2,0,0,0,2
1,5,normal,NaN,{W}{U}{B}{R}{G},1996 World Champion,*,NaN,[Legend],Cannot be the targets of spells or effects. Wo...,*,...,1,1,1,1,1,1,1,1,0,0
3,3,normal,NaN,{2}{W},AWOL,NaN,NaN,NaN,Remove target attacking creature from the game...,NaN,...,0,0,0,0,0,0,0,1,0,2
4,2,normal,NaN,{X}{1}{B},Abandon Hope,NaN,NaN,NaN,"As an additional cost to cast Abandon Hope, di...",NaN,...,0,0,0,0,0,0,1,0,1,1
6,4,normal,NaN,{3}{B},Abattoir Ghoul,3,NaN,[Zombie],First strike\nWhenever a creature dealt damage...,2,...,0,0,0,0,0,0,1,0,0,3


In [412]:
del data['manaCost']
del data['cmc']
data.head()

,layout,loyalty,name,power,starter,subtypes,text,toughness,type,types,...,red,green,blue,redCost,blueCost,greenCost,blackCost,whiteCost,xCost,colorlessCost
0,normal,NaN,"""Ach! Hans, Run!""",NaN,NaN,NaN,"At the beginning of your upkeep, you may say ""...",NaN,Enchantment,[Enchantment],...,1,1,0,2,0,2,0,0,0,2
1,normal,NaN,1996 World Champion,*,NaN,[Legend],Cannot be the targets of spells or effects. Wo...,*,Summon — Legend,[Creature],...,1,1,1,1,1,1,1,1,0,0
3,normal,NaN,AWOL,NaN,NaN,NaN,Remove target attacking creature from the game...,NaN,Instant,[Instant],...,0,0,0,0,0,0,0,1,0,2
4,normal,NaN,Abandon Hope,NaN,NaN,NaN,"As an additional cost to cast Abandon Hope, di...",NaN,Sorcery,[Sorcery],...,0,0,0,0,0,0,1,0,1,1
6,normal,NaN,Abattoir Ghoul,3,NaN,[Zombie],First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],...,0,0,0,0,0,0,1,0,0,3


In [413]:
uniq_subtypes = []

for i in data['subtypes']:
    if type(i) is list:
        for j in i:
            if j not in uniq_subtypes:
                uniq_subtypes.append(j)
    else:
        if i not in uniq_subtypes:
            uniq_subtypes.append(i)
                
uniq_subtypes = [x for x in uniq_subtypes if x is not np.nan]
uniq_subtypes

[u'Legend',
 u'Zombie',
 u'Gargoyle',
 u'Griffin',
 u'Human',
 u'Cleric',
 u'Monk',
 u'Aura',
 u'Demon',
 u'Horror',
 u'Elemental',
 u'Cephalid',
 u'Thrull',
 u'Cat',
 u'Assassin',
 u'Nightstalker',
 u'Specter',
 u'Hound',
 u'Shaman',
 u'Soldier',
 u'Warrior',
 u'Bird',
 u'Wizard',
 u'Knight',
 u'Equipment',
 u'Centaur',
 u'Spirit',
 u'Spider',
 u'Dragon',
 u'Ooze',
 u'Sliver',
 u'Insect',
 u'Construct',
 u'Lizard',
 u'Beast',
 u'Unicorn',
 u'Angel',
 u'Illusion',
 u'Goblin',
 u'Scout',
 u'Drone',
 u'Elf',
 u'Avatar',
 u'Archer',
 u'Ouphe',
 u'Artificer',
 u'Ally',
 u'Werewolf',
 u'Wall',
 u'Rogue',
 u'Advisor',
 u'Mercenary',
 u'Faerie',
 u'Ajani',
 u'Berserker',
 u'Giant',
 u'Phoenix',
 u'Horse',
 u'Djinn',
 u'Kirin',
 u'Leech',
 u'Troll',
 u'Orc',
 u'Spellshaper',
 u'Crocodile',
 u'Sphinx',
 u'Eldrazi',
 u'Golem',
 u'Myr',
 u'Siren',
 u'Kavu',
 u'Bear',
 u'Merfolk',
 u'Treefolk',
 u'Snake',
 u'Shapeshifter',
 u'Salamander',
 u'Kithkin',
 u'Rebel',
 u'Minotaur',
 u'Volver',
 u'Fish',

In [414]:
for i in uniq_subtypes:
    data[i] = [1 if hasItem(i,x) else 0 for x in data['subtypes']]
    
del data['subtypes']
data.head()

,layout,loyalty,name,power,starter,text,toughness,type,types,white,...,Thalakos,Lamia,Tibalt,Ship,Ugin,Venser,Vraska,Sponge,Mummy,Xenagos
0,normal,NaN,"""Ach! Hans, Run!""",NaN,NaN,"At the beginning of your upkeep, you may say ""...",NaN,Enchantment,[Enchantment],0,...,0,0,0,0,0,0,0,0,0,0
1,normal,NaN,1996 World Champion,*,NaN,Cannot be the targets of spells or effects. Wo...,*,Summon — Legend,[Creature],1,...,0,0,0,0,0,0,0,0,0,0
3,normal,NaN,AWOL,NaN,NaN,Remove target attacking creature from the game...,NaN,Instant,[Instant],1,...,0,0,0,0,0,0,0,0,0,0
4,normal,NaN,Abandon Hope,NaN,NaN,"As an additional cost to cast Abandon Hope, di...",NaN,Sorcery,[Sorcery],0,...,0,0,0,0,0,0,0,0,0,0
6,normal,NaN,Abattoir Ghoul,3,NaN,First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],0,...,0,0,0,0,0,0,0,0,0,0


In [415]:
np.unique(data['power'])

array([nan, u'*', u'*\xb2', u'-1', u'.5', u'0', u'1', u'1+*', u'1.5',
       u'10', u'11', u'12', u'13', u'15', u'2', u'2+*', u'2.5', u'3',
       u'3.5', u'4', u'5', u'6', u'7', u'8', u'9', u'99'], dtype=object)

In [416]:
def hasAsterisk(x):
    try:
        if '*' in x:
            return 1
        else:
            return 0
    except:
        return 1
    
def cleanPowerToughness(x):
    if x is np.nan:
        return 0
    try:
        int(x)
        return int(x)
    except:
        return 0
    
data['power_has_*'] = [hasAsterisk(x) for x in data['power']]
data['toughness_has_*'] = [hasAsterisk(x) for x in data['toughness']]
data['power'] = [cleanPowerToughness(x) for x in data['power']]
data['toughness'] = [cleanPowerToughness(x) for x in data['toughness']]
data.head()

,layout,loyalty,name,power,starter,text,toughness,type,types,white,...,Tibalt,Ship,Ugin,Venser,Vraska,Sponge,Mummy,Xenagos,power_has_*,toughness_has_*
0,normal,NaN,"""Ach! Hans, Run!""",0,NaN,"At the beginning of your upkeep, you may say ""...",0,Enchantment,[Enchantment],0,...,0,0,0,0,0,0,0,0,1,1
1,normal,NaN,1996 World Champion,0,NaN,Cannot be the targets of spells or effects. Wo...,0,Summon — Legend,[Creature],1,...,0,0,0,0,0,0,0,0,1,1
3,normal,NaN,AWOL,0,NaN,Remove target attacking creature from the game...,0,Instant,[Instant],1,...,0,0,0,0,0,0,0,0,1,1
4,normal,NaN,Abandon Hope,0,NaN,"As an additional cost to cast Abandon Hope, di...",0,Sorcery,[Sorcery],0,...,0,0,0,0,0,0,0,0,1,1
6,normal,NaN,Abattoir Ghoul,3,NaN,First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],0,...,0,0,0,0,0,0,0,0,0,0


In [417]:
data['starter'] = [1 if x is True else 0 for x in data['starter']]
data.head()

,layout,loyalty,name,power,starter,text,toughness,type,types,white,...,Tibalt,Ship,Ugin,Venser,Vraska,Sponge,Mummy,Xenagos,power_has_*,toughness_has_*
0,normal,NaN,"""Ach! Hans, Run!""",0,0,"At the beginning of your upkeep, you may say ""...",0,Enchantment,[Enchantment],0,...,0,0,0,0,0,0,0,0,1,1
1,normal,NaN,1996 World Champion,0,0,Cannot be the targets of spells or effects. Wo...,0,Summon — Legend,[Creature],1,...,0,0,0,0,0,0,0,0,1,1
3,normal,NaN,AWOL,0,0,Remove target attacking creature from the game...,0,Instant,[Instant],1,...,0,0,0,0,0,0,0,0,1,1
4,normal,NaN,Abandon Hope,0,0,"As an additional cost to cast Abandon Hope, di...",0,Sorcery,[Sorcery],0,...,0,0,0,0,0,0,0,0,1,1
6,normal,NaN,Abattoir Ghoul,3,0,First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],0,...,0,0,0,0,0,0,0,0,0,0


In [418]:
np.unique(data['layout'])

array([u'double-faced', u'flip', u'leveler', u'normal', u'split'], dtype=object)

In [419]:
uniq_layouts = list(set(data['layout']))
uniq_layouts

[u'split', u'double-faced', u'leveler', u'flip', u'normal']

In [420]:
for i in uniq_layouts:
    data["layout_"+i] = [1 if x==i else 0 for x in data['layout']]
del data['layout']
data.head()

,loyalty,name,power,starter,text,toughness,type,types,white,black,...,Sponge,Mummy,Xenagos,power_has_*,toughness_has_*,layout_split,layout_double-faced,layout_leveler,layout_flip,layout_normal
0,NaN,"""Ach! Hans, Run!""",0,0,"At the beginning of your upkeep, you may say ""...",0,Enchantment,[Enchantment],0,0,...,0,0,0,1,1,0,0,0,0,1
1,NaN,1996 World Champion,0,0,Cannot be the targets of spells or effects. Wo...,0,Summon — Legend,[Creature],1,1,...,0,0,0,1,1,0,0,0,0,1
3,NaN,AWOL,0,0,Remove target attacking creature from the game...,0,Instant,[Instant],1,0,...,0,0,0,1,1,0,0,0,0,1
4,NaN,Abandon Hope,0,0,"As an additional cost to cast Abandon Hope, di...",0,Sorcery,[Sorcery],0,1,...,0,0,0,1,1,0,0,0,0,1
6,NaN,Abattoir Ghoul,3,0,First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],0,1,...,0,0,0,0,0,0,0,0,0,1


In [421]:
np.unique(data['types'])

array([[u'Artifact'], [u'Artifact', u'Creature'], [u'Creature'],
       [u'Eaturecray'], [u'Enchant', u'Creature'], [u'Enchant', u'Player'],
       [u'Enchantment'], [u'Enchantment', u'Artifact'],
       [u'Enchantment', u'Creature'], [u'Instant'], [u'Planeswalker'],
       [u'Scariest', u'Creature', u"You'll", u'Ever', u'See'],
       [u'Sorcery'], [u'Tribal', u'Artifact'], [u'Tribal', u'Enchantment'],
       [u'Tribal', u'Instant'], [u'Tribal', u'Sorcery']], dtype=object)

In [422]:
text_keywords = "Deathtouch Defender Double Strike Enchant Equip First Strike Flash Flying \
Haste Hexproof Indestructible Lifelink Menace Prowess Reach Trample Vigilance Absorb Affinity\
Amplify Annihilator Aura Swap Awaken Banding Battle Cry Bestow Bloodthirst Bushido Buyback \
Cascade Champion Changeling Cipher Conspire Convoke Cumulative Upkeep Cycling Dash Delve Dethrone\
Devoid Devour Dredge Echo Entwine Epic Evoke Evolve Exalted Exploit Extort Fading Fear Flanking \
Flashback Forecast Fortify Frenzy Fuse Graft Gravestorm Haunt Hidden Agenda Hideaway Horsemanship\
Infect Ingest Intimidate Kicker Landwalk Level Up Living Weapon Madness Miracle Modular Morph Myriad\
Ninjutsu Offering Outlast Overload Persist Phasing Poisonous Protection Provoke Prowl Rampage\
Rebound Recover Reinforce Renown Replicate Retrace Ripple Scavenge Skulk Shadow Shroud Soulbond\
Soulshift Splice Split Second Storm Sunburst Surge Suspend Totem Armor Transfigure Transmute\
Tribute Undying Unearth Unleash Vanishing Wither Activate Attach Cast Counter Destroy Discard\
Exchange Exile Fight Play Regenerate Reveal Sacrifice Scry Search Shuffle Tap Untap"

text_keywords.split()

['Deathtouch',
 'Defender',
 'Double',
 'Strike',
 'Enchant',
 'Equip',
 'First',
 'Strike',
 'Flash',
 'Flying',
 'Haste',
 'Hexproof',
 'Indestructible',
 'Lifelink',
 'Menace',
 'Prowess',
 'Reach',
 'Trample',
 'Vigilance',
 'Absorb',
 'AffinityAmplify',
 'Annihilator',
 'Aura',
 'Swap',
 'Awaken',
 'Banding',
 'Battle',
 'Cry',
 'Bestow',
 'Bloodthirst',
 'Bushido',
 'Buyback',
 'Cascade',
 'Champion',
 'Changeling',
 'Cipher',
 'Conspire',
 'Convoke',
 'Cumulative',
 'Upkeep',
 'Cycling',
 'Dash',
 'Delve',
 'DethroneDevoid',
 'Devour',
 'Dredge',
 'Echo',
 'Entwine',
 'Epic',
 'Evoke',
 'Evolve',
 'Exalted',
 'Exploit',
 'Extort',
 'Fading',
 'Fear',
 'Flanking',
 'Flashback',
 'Forecast',
 'Fortify',
 'Frenzy',
 'Fuse',
 'Graft',
 'Gravestorm',
 'Haunt',
 'Hidden',
 'Agenda',
 'Hideaway',
 'HorsemanshipInfect',
 'Ingest',
 'Intimidate',
 'Kicker',
 'Landwalk',
 'Level',
 'Up',
 'Living',
 'Weapon',
 'Madness',
 'Miracle',
 'Modular',
 'Morph',
 'MyriadNinjutsu',
 'Offering',
 '

In [423]:
for keyword in text_keywords.split():
    data['keyword_'+keyword] = [1 if x==keyword else 0 for x in data['text']]
    
data.head()

,loyalty,name,power,starter,text,toughness,type,types,white,black,...,keyword_Fight,keyword_Play,keyword_Regenerate,keyword_Reveal,keyword_Sacrifice,keyword_Scry,keyword_Search,keyword_Shuffle,keyword_Tap,keyword_Untap
0,NaN,"""Ach! Hans, Run!""",0,0,"At the beginning of your upkeep, you may say ""...",0,Enchantment,[Enchantment],0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,1996 World Champion,0,0,Cannot be the targets of spells or effects. Wo...,0,Summon — Legend,[Creature],1,1,...,0,0,0,0,0,0,0,0,0,0
3,NaN,AWOL,0,0,Remove target attacking creature from the game...,0,Instant,[Instant],1,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Abandon Hope,0,0,"As an additional cost to cast Abandon Hope, di...",0,Sorcery,[Sorcery],0,1,...,0,0,0,0,0,0,0,0,0,0
6,NaN,Abattoir Ghoul,3,0,First strike\nWhenever a creature dealt damage...,2,Creature — Zombie,[Creature],0,1,...,0,0,0,0,0,0,0,0,0,0


In [424]:
del data['text']
data.head()

,loyalty,name,power,starter,toughness,type,types,white,black,red,...,keyword_Fight,keyword_Play,keyword_Regenerate,keyword_Reveal,keyword_Sacrifice,keyword_Scry,keyword_Search,keyword_Shuffle,keyword_Tap,keyword_Untap
0,NaN,"""Ach! Hans, Run!""",0,0,0,Enchantment,[Enchantment],0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,NaN,1996 World Champion,0,0,0,Summon — Legend,[Creature],1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,NaN,AWOL,0,0,0,Instant,[Instant],1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Abandon Hope,0,0,0,Sorcery,[Sorcery],0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,NaN,Abattoir Ghoul,3,0,2,Creature — Zombie,[Creature],0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [425]:
uniq_types = []
for i in np.unique(data['types']):
    for j in i:
        if j not in uniq_types:
            uniq_types.append(j)
            
for uniq_type in uniq_types:
    data['type_'+uniq_type] = [int(hasItem(uniq_type,x)) for x in data['types']]
data.head()

,loyalty,name,power,starter,toughness,type,types,white,black,red,...,type_Player,type_Enchantment,type_Instant,type_Planeswalker,type_Scariest,type_You'll,type_Ever,type_See,type_Sorcery,type_Tribal
0,NaN,"""Ach! Hans, Run!""",0,0,0,Enchantment,[Enchantment],0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,NaN,1996 World Champion,0,0,0,Summon — Legend,[Creature],1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,NaN,AWOL,0,0,0,Instant,[Instant],1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,NaN,Abandon Hope,0,0,0,Sorcery,[Sorcery],0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,NaN,Abattoir Ghoul,3,0,2,Creature — Zombie,[Creature],0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [426]:
del data['types']

In [427]:
np.unique(data['loyalty'])

array([nan, 2, 3, 4, 5, 6, 7], dtype=object)

In [428]:
def removeNan(x):
    if x is np.nan:
        return 0
    else:
        return int(x)
    
data['loyalty'] = [removeNan(x) for x in data['loyalty']]
data.head()

,loyalty,name,power,starter,toughness,type,white,black,red,green,...,type_Player,type_Enchantment,type_Instant,type_Planeswalker,type_Scariest,type_You'll,type_Ever,type_See,type_Sorcery,type_Tribal
0,0,"""Ach! Hans, Run!""",0,0,0,Enchantment,0,0,1,1,...,0,1,0,0,0,0,0,0,0,0
1,0,1996 World Champion,0,0,0,Summon — Legend,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,AWOL,0,0,0,Instant,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,0,Abandon Hope,0,0,0,Sorcery,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
6,0,Abattoir Ghoul,3,0,2,Creature — Zombie,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [429]:
def isLegendary(x):
    if "Legendary" in x:
        return 1
    else:
        return 0
    
data['isLegendary'] = [isLegendary(x) for x in data['type']]
del data['type']
del data['name']
data.head()

,loyalty,power,starter,toughness,white,black,red,green,blue,redCost,...,type_Enchantment,type_Instant,type_Planeswalker,type_Scariest,type_You'll,type_Ever,type_See,type_Sorcery,type_Tribal,isLegendary
0,0,0,0,0,0,0,1,1,0,2,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
6,0,3,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [430]:
data.to_csv("cleaned_AllCards.csv")